In [1]:
import tensorflow as tf
import numpy as np

In [2]:
!cat '/content/drive/My Drive/MasterThesis/Datasets/Offensive2020/trainDev/NOT_OFF/1000.txt'

@USER @USER ان شاء الله يا غالي حبيبي يا محمد<LF>ودايما متجمعين علي عشق الكيان	NOT_OFF	NOT_HS


In [3]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/My Drive/MasterThesis/Datasets/Offensive2020/trainDev",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337
   
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/My Drive/MasterThesis/Datasets/Offensive2020/trainDev",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/My Drive/MasterThesis/Datasets/Offensive2020/test", batch_size=batch_size
)

print(
    "Number of batches in raw_train_ds: %d"
    % tf.data.experimental.cardinality(raw_train_ds)
)
print(
    "Number of batches in raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds)
)
print(
    "Number of batches in raw_test_ds: %d"
    % tf.data.experimental.cardinality(raw_test_ds)
)

Found 8000 files belonging to 2 classes.
Using 6400 files for training.
Found 8000 files belonging to 2 classes.
Using 1600 files for validation.
Found 2000 files belonging to 2 classes.
Number of batches in raw_train_ds: 200
Number of batches in raw_val_ds: 50
Number of batches in raw_test_ds: 63


In [ ]:
# It's important to take a look at your raw data to ensure your normalization
# and tokenization will work as expected. We can do that by taking a few
# examples from the training set and looking at them.
# This is one of the places where eager execution shines:
# we can just evaluate these tensors using .numpy()
# instead of needing to evaluate them in a Session/Graph context.
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(10):
        print(text_batch.numpy()[i].decode('utf-8').strip())
        print(label_batch.numpy()[i])
        print('--------------------------------')

RT @USER: يا رب يا عزيز يا جبار .. انك القادر على كل شيء .. يا رب فرحه اتحاديه تُنسينا كل الهموم والمشاكل الي صارت هالموسم يا رب العباد…	NOT_OFF	NOT_HS
0
--------------------------------
@USER @USER يا جامع يا رقيب يا رب تلقاها	NOT_OFF	NOT_HS
0
--------------------------------
راهنت عليك ولم اخسر ❤️❤️<LF>يا وحش يا جلاد يا كبير<LF>يا مرعب يا قناص يا يصياد<LF>🖤💛🖤💛🖤💛🖤💛<LF>#الاتحاد_النصر URL	NOT_OFF	NOT_HS
0
--------------------------------
RT @USER: كانت أيام يا وطني<LF>زي الأحلام يا وطني 🎶	NOT_OFF	NOT_HS
0
--------------------------------
تمثيل يا رزان تمثيل يا رزان تمثيل يا رزان تمثيل يا رزان تمثيل يا رزان تمثيل يا رزان	NOT_OFF	NOT_HS
0
--------------------------------
يا HBO يا شرموطة تنزلي الحلقة الأولى من GOT وانا معرفش يا لبوة .. أخص عليكي شركة إنتاج وثخة 😁😂	OFF	NOT_HS
1
--------------------------------
@USER يا سلام عليك يا عميد 💙💛 سلطان بطل وراح يبقى بطل مهما حصل حنا عيال النصر بكل الاحوال	NOT_OFF	NOT_HS
0
--------------------------------
@USER @USER يا دحابشه يا كرام ...<LF>الجنو

In [4]:
# xx = re.sub(r"(.)\1+", r"\1", "سلاااااام")
xx=tf.strings.regex_replace("سلااااااااااااااااام", "r'(.)\1+'", "r'\1\1'")
for i in xx.numpy():
    tf.compat.as_str_any(i)


In [11]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re
text_cleaning_re = "[a-zA-Z]|\d+|[٠١٢٣٤٥٦٧٨٩]|[.#،<>:@,\\-_”“٪ًَ]"

def custom_standardization(text):
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", str(text))
    # text = text.replace('وو', 'و')
    # text = text.replace('يي', 'ي')
    # text = text.replace('اا', 'ا')
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    #remove longation
    text = re.sub(r'(.)\1+', r'\1\1', str(text)) 
    text = re.sub("[إأآا]", "ا", str(text))
    text = re.sub(r"([?!¿])", r" \1 ", str(text))
    text = re.sub("ى", "ي", str(text))
    # text = re.sub("ؤ", "ء", text)
    # text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", str(text))


    # x = tf.strings.regex_replace(input_data,"r'(.)\1+'", "r'\1'")
    # stripped_html = re.sub(r'(.)\1+', r'\1',str(input_data, 'utf-8'))
    # stripped_html = tf.strings.regex_replace(stripped_html, "<LF>", " ")
    # stripped_html = tf.strings.regex_replace(stripped_html, "[a-zA-Z]|\d+|[٠١٢٣٤٥٦٧٨٩]", " ")
    # stripped_html = tf.strings.regex_replace(stripped_html, "[.،,\\-_”“٪ًَ]", " ")
    
    # stripped_html = tf.strings.regex_replace(stripped_html, "[إأآا]", "ا")
    # stripped_html = tf.strings.regex_replace(stripped_html, "ة", "ه")
  
    return tf.strings.regex_replace(
       text, "[%s]" % re.escape(string.punctuation), ""
    )

In [12]:
custom_standardization('مشكوووووور')

<tf.Tensor: shape=(), dtype=string, numpy=b'\xd9\x85\xd8\xb4\xd9\x83\xd9\x88\xd9\x88\xd8\xb1'>

In [13]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re
# @tf.function()
def custom_standardization(input_data):
    # lowercase = tf.strings.lower(input_data)
    # input_data = tf.ragged.strings.bytes_split(input_data)
    
    # input_data = tf.constant(input_data)
    # text_chars = tf.constant([ord(char) for char in input_data])

    # stripped_html = re.sub(r'(.)\1+', r'\1',input_data.numpy() )


    # stripped_html = re.sub(r'(.)\1+', r'\1',str(input_data.eval())) 
    # stripped_html = re.sub(r'(.)\1+', r'\1', str(lowercase))
    # z=tf.strings.regex_replace(lowercase,  "(.)\1+", "\1")


    # x= tf.compat.as_str_any(input_data)
    stripped_html = re.sub(r'(.)\1+', r'\1',input_data)
    stripped_html = tf.strings.regex_replace(stripped_html, "<LF>", " ")
    stripped_html = tf.strings.regex_replace(stripped_html, "[a-zA-Z]|\d+|[٠١٢٣٤٥٦٧٨٩]", " ")
    stripped_html = tf.strings.regex_replace(stripped_html, "[.،,\\-_”“٪ًَ]", " ")
    
    stripped_html = tf.strings.regex_replace(stripped_html, "[إأآا]", "ا")
    stripped_html = tf.strings.regex_replace(stripped_html, "ة", "ه")
    # stripped_html = re.sub(r'(.)\1+', r'\1', tf.compat.as_str(stripped_html,encoding='utf-8'))

    # corpus =[]
    # for i in range(len(input_data)):
    # lowercase = re.sub(r"(.)\1+", r"\1", str(input_data))
    # lowercase=tf.strings.unicode_decode(lowercase,input_encoding='UTF-8')
        # tweet = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*\s"," ", tweet)
        # tweet = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*$"," ", tweet)
        # tweet = tweet.lower()
        # corpus.append(tweet)
    # lowercase = tf.strings.lower(input_data)
    # # lowercase=lowercase.decode('utf-8')
    # # stripped_html = re.sub(r'(.)\1+', 'r'\1', str(lowercase))
    # stripped_html = tf.strings.regex_replace(lowercase, "<LF>", " ")
    # stripped_html = tf.strings.regex_replace(stripped_html, "[a-zA-Z]|\d+|[٠١٢٣٤٥٦٧٨٩]", " ")
    # stripped_html = tf.strings.regex_replace(stripped_html, "[.،,\\_-”“٪ًَ]", " ")
    
    # stripped_html = tf.strings.regex_replace(stripped_html, "[إأآا]", "ا")
    # stripped_html = tf.strings.regex_replace(stripped_html, "ة", "ه")
    # stripped_html=tf.strings.regex_replace(stripped_html, "[(\U0001F600-\U0001F92F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F190-\U0001F1FF|\U00002702-\U000027B0|\U0001F926-\U0001FA9F|\u200d|\u2640-\u2642|\u2600-\u2B55|\u23cf|\u23e9|\u231a|\ufe0f)|\u2069|\u2066]+", " ")
    # stripped_html = tf.strings.regex_replace(stripped_html, "r'(.)\1+'", "r'\1\1'")

    return tf.strings.regex_replace(
       stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )
   
#max_features = 20000 #13% 
max_features = 50000 #13% 
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

text_ds = raw_train_ds.map(lambda x, y: x)
# text_ds
vectorize_layer.adapt(text_ds)


TypeError: ignored

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

In [ ]:
vocab = np.array(vectorize_layer.get_vocabulary())
vocab[:1000]

In [ ]:
vectorize_layer

In [ ]:
encoded_example = vectorize_layer(text_batch)[:100].numpy()
encoded_example

In [ ]:
for n in range(5):
  print("Original: ", text_batch[n].numpy().decode('utf-8').strip())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

## Build a model 

In [ ]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionalityl
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
# x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
# x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

##################### salam here Add Bi lstm layer #########################

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## Train the model

In [ ]:
from keras.callbacks import EarlyStopping
epochs = 30

# Fit the model using the train and test datasets.
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs,
                    callbacks=[EarlyStopping(monitor='val_loss',patience=5)])




## Evaluate the model on the test set


In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))


## plotlib

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

#epochs_range = range(22)

plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# !pip install tensorflow.js